# 0. 기본 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import random
import os
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
sns.set(font="Malgun Gothic")
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# 데이터 분할 및 평가 지표

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

# XGBoost 라이브러리

In [3]:
import xgboost as xgb
from xgboost import plot_importance 
from xgboost import XGBRegressor

# 파라미터 튜닝 함수

In [4]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import optuna

# 빈도테이블 생성

In [5]:
def count(x):
    data1=pd.DataFrame(x.value_counts()).reset_index()
    data1.columns=['category','빈도수']
    return data1

# Barplot 생성

In [6]:
def bargraph(x,y,x_label,y_label,figsize1,figsize2,data):
    plt.figure(figsize=(figsize1, figsize2))
    if data[x].dtypes=='int64':
        data[x]=data[x].astype('str')
    data=data.sort_values(by=y,ascending=False).reset_index()
    
    for index, row in data.iterrows():
        g=sns.barplot(x,y,data=data)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        g.text(index,row[y],row[y],color='black',ha='center')

# 성능 지표 추출

In [7]:
# 성능지표 추출
def Evaluation_metric(actual,pred):
    print(f'MAE: {round(mean_absolute_error(actual,pred),2)}')
    print(f'MSE: {round(mean_squared_error(actual,pred),2)}')
    print(f'RMSE: {round(sqrt(mean_squared_error(actual,pred)),2)}')
    #print(f'RMSPE: {round(sqrt(mean_squared_error(actual,pred))/np.mean(actual)*100,2)}%')
    print(f'RMSPE: {round(np.sqrt(np.mean(((actual - pred) / actual) ** 2)) * 100, 2)}%')

# 1. 시드 고정

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

# 2. 데이터 불러오기

In [9]:
# Train, Valid, Test Split용 데이터
data=pd.read_csv(os.listdir()[10],encoding='EUC-KR')
# 지표 확인용 데이터
data_real=pd.read_csv(os.listdir()[10],encoding='EUC-KR')
# 배정신청인원 0명 제거
data=data.query('합계!=0')
#필요 컬럼 추출
selected_columns_1=['구분','합계', '작물 종류','농지면적(실제경작)','전년대비농경체증감률','고령농경체비율','전년도이탈인원','전년도활용여부']
selected_columns_2=['비고', '지자체명_시도', '지자체명_시군구', '구분', '농업경영체','합계', '작물 종류','농지면적(실제경작)','전년대비농경체증감률','고령농경체비율','전년도이탈인원','전년도활용여부'] #원본 데이터 컬럼
data=data[selected_columns_1]
data_real=data_real[selected_columns_2]
# 농지면적(실제경작) 컬럼명 변경
data.rename(columns={'농지면적(실제경작)':'농지면적','작물 종류':'작물종류'},inplace=True)

# 3. 이상치 제거: 농지 면적, 합계

In [10]:
data=data.query('0.04<농지면적<2.5')
data=data.query('합계<11')

# 4. 데이터 분할 Train, Valid, Test

In [11]:
# 작물 종류 기준 층화 추출 Train Test 비율 8:2
X=data.drop(columns=['합계']) #독립변수 Set
Y=data['합계'] #Target 변수 set

# Train, Test 분할
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,stratify=X['작물종류'],random_state=42)

# 2차 Train, Validation 분할
X_train, X_valid, Y_train, Y_valid=train_test_split(X_train,Y_train,test_size=0.2,stratify=X_train['작물종류'],random_state=42)

# 5. 연속형 변수 정규화 & 범주형 변수 라벨 인코딩 진행

In [12]:
# 연속형 변수 정규화
min_max_scaler=MinMaxScaler()
for i in X_train.columns:
    if (X_train[i].dtypes!='object'):
        X_train[i]=min_max_scaler.fit_transform(X_train[[i]])
        X_valid[i]=min_max_scaler.transform(X_valid[[i]])
        X_test[i]=min_max_scaler.transform(X_test[[i]])
    else:
        pass

In [13]:
# 범주형 변수 라벨 인코딩
label_encoder=LabelEncoder()
for i in X_train.columns:
    if X_train[i].dtypes=='object':
        X_train[i]=label_encoder.fit_transform(X_train[i])
        X_valid[i]=label_encoder.transform(X_valid[i])
        X_test[i]=label_encoder.transform(X_test[i])
    else:
        pass

# 6. XGBoost parameter Tuning

- 주요 파라미터 설명

- gamma(값이 클수록 과적합을 방지해줌)=> [0,∞], 기본값=0=> 손실감소량에 대한 임계치임
ex) gamma가 2일때 loss가 2이하면 더 이상 트리 분할 X

- max_depth= Tree maximum depth 값
ex) 최대 가지 개수

- subsample= 데이터 샘플링 비율
ex) 0.8로 설정하면 각 트리가 학습할 때 전체 데이터의 80% 사용 => 과적합 방지
-------------------------------------------------------------------------------------------------------------


- 초기 파라미터
- dart부스터 학습률 0.05, 한트리의 최대 깊이 8, 반복 학습 횟수 200회로 진행
- XGBRegressor(booster='dart',learning_rate=0.05, max_depth=8, n_estimator=200)

-------------------------------------------------------------------------------------------------------------

- 파라미터 튜닝

- 학습률 0.05~0.01

- gamma 0~2

- max_depth 6~9

- subsample 0.5~1

- 반복수 100~300

# 7. 초기 기본 모델

농업경영체별 평가지표 결과
MAE: 1.31
MSE: 3.4
RMSE: 1.84
RMSPE: 138.54%
----------------------------------------------------------------------------------------------------
배정신청인원 10명 이상 시군구 단위 평가지표
MAE: 7.53
MSE: 103.81
RMSE: 10.19
RMSPE: 20.4%
----------------------------------------------------------------------------------------------------
변수중요도 Graph


<AxesSubplot:title={'center':'Feature importance'}, xlabel='F score', ylabel='Features'>

In [20]:
# XGB regressor 모델 구축
# dart부스터 학습률 0.05, 한트리의 최대 깊이 8, 반복 학습 횟수 200회로 진행=> 파라미터 튜닝 X
model_reg=XGBRegressor(booster='dart',learning_rate=0.05, max_depth=10, n_estimators=200)
# 모델 적용
model_reg.fit(X_train,Y_train)
#예측값 
pred=model_reg.predict(X_test)
# 인원 수 예측이기에 예측된 결과에 반올림 적용
pred=pd.DataFrame(pred,columns=['예측값'])
#반올림
pred['예측값']=round(pred['예측값'],0)
#평가지표
print('농업경영체별 평가지표 결과')
Evaluation_metric(Y_test,pred=pred['예측값'])
print('-'*100)

test_index=X_test.index.tolist()
data_test=data_real.loc[test_index]
data_test=data_test.reset_index()
data_test.drop(columns=['index'],inplace=True)
#예측값
data_test['predict']=pred
result=data_test.groupby(['지자체명_시도','지자체명_시군구'])[['합계','predict']].sum().reset_index()
result_1=result.query('합계>10')
print('배정신청인원 10명 이상 시군구 단위 평가지표')
Evaluation_metric(result_1['합계'],pred=result_1['predict'])
print('-'*100)
print('변수중요도 Graph')
# 변수 중요도 확인
plot_importance(model_reg)

# 8. Bayesian Optimization 튜닝

In [21]:
# 목적 함수 정의
results = {
    'max_depth': [],
    'gamma': [],
    'subsample': [],
    'n_estimator': [],
    'learning_rate': [],
    'target': []  # 최적화 결과인 target 값 저장
}

def xgb_cv(max_depth, gamma, subsample, n_estimator, learning_rate):
    params = {
        'eval_metric': 'rmse',
        'max_depth': int(max_depth),
        'gamma': gamma,
        'subsample': subsample,
        'n_estimator': int(n_estimator),
        'learning_rate': learning_rate,
        'booster': 'dart'
    }
    
    # XGBoost Regressor 모델 초기화
    model_reg = xgb.XGBRegressor(**params)
    
    #model Train 학습
    model_reg.fit(X_train,Y_train)
    #model validation
    scores = -cross_val_score(model_reg, X_valid, Y_valid, cv=5, scoring='neg_mean_squared_error').mean()
    results['max_depth'].append(max_depth)
    results['gamma'].append(gamma)
    results['subsample'].append(subsample)
    results['n_estimator'].append(n_estimator)
    results['learning_rate'].append(learning_rate)
    results['target'].append(scores)
    return scores

# Bayesian Optimization 수행
xgbBO = BayesianOptimization(
    xgb_cv,
    {'max_depth': (8, 10),
     'gamma': (1, 2),
     'subsample': (0.85, 1.0),
     'n_estimator': (200, 300),
     'learning_rate': (0.04,0.05)}
)

# 최적화
xgbBO.maximize(init_points=10, n_iter=15)

idx_of_min=results['target'].index(min(results['target']))
min_pam={'booster':'dart'}
for key,value in results.items():
    if key=='target':
        pass
    else:
        if (key=='max_depth') or (key=='n_estimator'):
            min_pam[key]=int(round(value[idx_of_min],0))
        else:
            min_pam[key]=value[idx_of_min]
            
print(f'최종 파라미터는 {min_pam}입니다')

model_reg = xgb.XGBRegressor(**min_pam)
model_reg.fit(X_train,Y_train)

#예측값 
pred=model_reg.predict(X_test)
# 인원 수 예측이기에 예측된 결과에 반올림 적용
pred=pd.DataFrame(pred,columns=['예측값'])
#반올림
pred['예측값']=round(pred['예측값'],0)
#평가지표
print('농업경영체별 평가지표 결과')
Evaluation_metric(Y_test,pred=pred['예측값'])
print('-'*100)

test_index=X_test.index.tolist()
data_test=data_real.loc[test_index]
data_test=data_test.reset_index()
data_test.drop(columns=['index'],inplace=True)
#예측값
data_test['predict']=pred
result=data_test.groupby(['지자체명_시도','지자체명_시군구'])[['합계','predict']].sum().reset_index()
result_1=result.query('합계>10')
print('배정신청인원 10명 이상 시군구 단위 평가지표')
Evaluation_metric(result_1['합계'],pred=result_1['predict'])

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
| 1         | 3.462     | 1.853     | 0.04178   | 8.704     | 243.2     | 0.9208    |
| 2         | 3.461     | 1.245     | 0.04029   | 8.523     | 237.6     | 0.9467    |
| 3         | 3.483     | 1.879     | 0.04899   | 9.677     | 279.2     | 0.9681    |
| 4         | 3.462     | 1.178     | 0.04049   | 8.437     | 274.5     | 0.961     |
| 5         | 3.496     | 1.335     | 0.04079   | 9.137     | 262.5     | 0.935     |
| 6         | 3.436     | 1.999     | 0.04033   | 8.206     | 299.9     | 0.8522    |
| 7         | 3.47      | 1.491     | 0.04459   | 9.53      | 254.9     | 0.9479    |
| 8         | 3.458     | 1.617     | 0.04878   | 8.212     | 240.4     | 0.869     |
| 9         | 3.463     | 1.348     | 0.04834   | 8.53      | 254.9     | 0.8957    |
| 10        | 3.516     | 1.206     | 0.04416   | 9.76

# 9. OPTUNA 라이브러리 활용 파라미터 튜닝

파라미터 튜닝 코드

- optuna.trial.Trial.suggest_categorical() : 리스트 범위 내에서 값을 선택한다.
- optuna.trial.Trial.suggest_int() : 범위 내에서 정수형 값을 선택한다.
- optuna.trial.Trial.suggest_float() : 범위 내에서 소수형 값을 선택한다.
- optuna.trial.Trial.suggest_uniform() : 범위 내에서 균일분포 값을 선택한다.
- optuna.trial.Trial.suggest_discrete_uniform() : 범위 내에서 이산 균일분포 값을 선택한다.
- optuna.trial.Trial.suggest_loguniform() : 범위 내에서 로그 함수 값을 선택한다.

In [22]:
def objective(trial):
    params={
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth',8,10), #8~10 정수중 택 1
        'gamma': trial.suggest_loguniform('gamma',1,2),
        'subsample': trial.suggest_categorical('subsample',[0.8,0.85,0.90,0.95,1]),
        'n_estimator': trial.suggest_int('n_estimator',200,400),
        'learning_rate': trial.suggest_float('learning_rate',0.03,0.05),
        'booster': 'dart'   
    }
    # 학습 모델 생성
    model_reg=xgb.XGBRegressor(**params)
    
    #트레인 셋 학습
    model_reg.fit(X_train,Y_train)
    
    #검증 데이터 셋으로 모델 검증
    score = mean_absolute_error(model_reg.predict(X_valid), Y_valid)
    return score

study=optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150)

# 최적화된 파라미터 값 확인
best_params = study.best_params
print(best_params)

model_reg = xgb.XGBRegressor(**best_params)
model_reg.fit(X_train,Y_train)

#예측값 
pred=model_reg.predict(X_test)
# 인원 수 예측이기에 예측된 결과에 반올림 적용
pred=pd.DataFrame(pred,columns=['예측값'])
#반올림
pred['예측값']=round(pred['예측값'],0)
#평가지표
print('농업경영체별 평가지표 결과')
Evaluation_metric(Y_test,pred=pred['예측값'])
print('-'*100)

test_index=X_test.index.tolist()
data_test=data_real.loc[test_index]
data_test=data_test.reset_index()
data_test.drop(columns=['index'],inplace=True)
#예측값
data_test['predict']=pred
result=data_test.groupby(['지자체명_시도','지자체명_시군구'])[['합계','predict']].sum().reset_index()
result_1=result.query('합계>10')
print('배정신청인원 10명 이상 시군구 단위 평가지표')
Evaluation_metric(result_1['합계'],pred=result_1['predict'])

[I 2023-11-28 10:55:06,374] A new study created in memory with name: no-name-8e816f46-2d7d-4769-9157-1f24aa94b494
[I 2023-11-28 10:55:07,993] Trial 0 finished with value: 1.318716602198087 and parameters: {'max_depth': 9, 'gamma': 1.8718909399939394, 'subsample': 0.9, 'n_estimator': 376, 'learning_rate': 0.04490859791064177}. Best is trial 0 with value: 1.318716602198087.
[I 2023-11-28 10:55:09,508] Trial 1 finished with value: 1.3123488151910798 and parameters: {'max_depth': 8, 'gamma': 1.2924894794735549, 'subsample': 0.9, 'n_estimator': 295, 'learning_rate': 0.04921685971869262}. Best is trial 1 with value: 1.3123488151910798.
[I 2023-11-28 10:55:11,133] Trial 2 finished with value: 1.3221926973241993 and parameters: {'max_depth': 8, 'gamma': 1.7310296074825466, 'subsample': 1, 'n_estimator': 206, 'learning_rate': 0.048349351042270686}. Best is trial 1 with value: 1.3123488151910798.
[I 2023-11-28 10:55:12,921] Trial 3 finished with value: 1.3257431650871978 and parameters: {'max_de

{'max_depth': 8, 'gamma': 1.4275441794079136, 'subsample': 0.9, 'n_estimator': 212, 'learning_rate': 0.049675497125767604}
농업경영체별 평가지표 결과
MAE: 1.27
MSE: 3.07
RMSE: 1.75
RMSPE: 131.27%
----------------------------------------------------------------------------------------------------
배정신청인원 10명 이상 시군구 단위 평가지표
MAE: 8.56
MSE: 146.64
RMSE: 12.11
RMSPE: 21.24%
